# Gather Evidence Using PaperQA2

![Agent with scientific knowledge tool](../../../static/agent-gather-evidence.png)

The lab introduces sophisticated RAG techniques including

- Re-ranking of retrieved passages for relevance
- Contextual summarization of evidence
- Precise citation tracking with token count monitoring
- Cost estimation for different model configurations

This lab uses PaperQA2, a sophisticated retrieval-augmented generation (RAG) agent developed by FutureHouse that specializes in scientific literature analysis. It uses a technique called "reranking and contextual summarization" or RCS to improve the quality of retrieved evidence. This proces begins with a standard top-k dense vector rerieval step using embedding vectors to identify potentially relevant document chunks. High-scoring chunks are then summarized using a LLM and reranked to ensure that only the most relevant text influences the final answer. PaperQA2 demonstrated superior performance, achieving 85.2% precision in question answering tasks compared to 73.8% human performance [1].

In this lab, you will explore the impact of advanced RAG techniques on the quality of the agentic response. You will also experiment with different LLMs and observe the impact on output quality and inference cost. Finally you will build a incorporate PaperQA2 into a `generate_evidence` tool and add it to a Strands agent deployed on Bedrock AgentCore.

## Sources

1. Skarlinski, Michael D., et al. "Language agents achieve superhuman synthesis of scientific knowledge." arXiv preprint arXiv:2409.13740, 26 Sept. 2024, doi.org/10.48550/arXiv.2409.13740.

## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit paper-qa

## 2. Experiment with PaperQA2

### 2.1. Basic usage

Start by downloading download an example paper from PubMed Central: 

Rodriguez, Patricia J., et al. "Semaglutide vs Tirzepatide for Weight Loss in Adults With Overweight or Obesity." JAMA Internal Medicine, vol. 184, no. 9, 8 July 2024, pp. 1056-1064, doi:10.1001/jamainternmed.2024.2525.


In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
from pathlib import Path


s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

pmc_id = "PMC9438179"
bucket_name = "pmc-oa-opendata"
object_key = f"oa_comm/txt/all/{pmc_id}.txt"

# Create papers directory if it doesn't exist
papers_dir = Path(f"my_papers/{pmc_id}/txt")
papers_dir.mkdir(parents=True, exist_ok=True)

local_file_path = f"my_papers/{pmc_id}/txt/{pmc_id}.txt"
s3.download_file(bucket_name, object_key, local_file_path)

In [ ]:
from helpers import pretty_print_paperqa_results
from paperqa import Settings, ask
from paperqa.settings import (
    AgentSettings,
    ParsingSettings,
    IndexSettings,
    AnswerSettings,
)

LLM = "global.anthropic.claude-sonnet-4-20250514-v1:0"

answer_response = await ask(
    "How safe and effective are GLP-1 drugs for long term use?",
    settings=Settings(
        llm=LLM,
        summary_llm=LLM,
        agent=AgentSettings(
            agent_llm=LLM,
            index=IndexSettings(
                index_directory="my_papers/PMC9438179/index",
                paper_directory="my_papers/PMC9438179/txt",
            ),
        ),
        embedding="bedrock/amazon.titan-embed-text-v2:0",
        parsing=ParsingSettings(use_doc_details=False),
    ),
)


pretty_print_paperqa_results(answer_response)

### 2.2. Cost Optimization with Multiple LLMs

PaperQA supports a variety of settings to optimize price and performance. In particular, you can use different LLMs for various tasks:

- `llm`: LLM for general use including metadata inference and answer generation. Should be 'best' LLM. Uses include:
    1. Inferring citation information from documents when left unspecified
    1. Extracting title, DOI, and authors from citation information when left unspecified
    1. Optionally injecting pre-answer information
    1. Generating an answer given evidence
    1. Optionally injecting post-answer information 
- `summary_llm`: LLM for creating contextual summaries 
- `agent_llm`: LLM inside the agent making tool selections.

In [ ]:
from helpers import pretty_print_paperqa_results
from paperqa import Settings, ask
from paperqa.settings import (
    AgentSettings,
    ParsingSettings,
    IndexSettings,
    AnswerSettings,
)

LLM = "global.anthropic.claude-sonnet-4-20250514-v1:0"
SUMMARY_LLM = "bedrock/openai.gpt-oss-120b-1:0"
AGENT_TYPE = "fake"
EVIDENCE_K = 5
EVIDENCE_SUMMARY_LENGTH = "25 to 50 words"

answer_response = await ask(
    "What is PaperQA?",
    settings=Settings(
        llm=LLM,
        summary_llm=SUMMARY_LLM,
        agent=AgentSettings(
            agent_llm=LLM,
            index=IndexSettings(
                index_directory="my_papers/paperqa/index",
                paper_directory="my_papers/paperqa/txt",
            ),
            agent_type=AGENT_TYPE,
        ),
        embedding="bedrock/amazon.titan-embed-text-v2:0",
        parsing=ParsingSettings(use_doc_details=False),
        answer=AnswerSettings(
            answer_max_sources=1,
            evidence_k=EVIDENCE_K,
            evidence_summary_length="about 50 words",
        ),
    ),
)

pretty_print_paperqa_results(answer_response)

By using a different model for the evidence summarization step, we were able to significantly reduce the estimated cost!

## 3. Create Strands Agent and tool

### 3.1. Call Gather Evidence Tool Directly

There's an example of how to add PaperQA to a Strands tool in `gather_evidence.py`. Try it out directly and look at the results.

In [ ]:
from strands import Agent
from gather_evidence import gather_evidence_tool

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
agent = Agent(tools=[gather_evidence_tool], model=MODEL_ID)

agent.tool.gather_evidence_tool(
    pmcid="PMC9438179",
    question="How safe and effective are GLP-1 drugs for long term use?",
)

The `gather_evidence` tool returns a structured evidence record with some helpful information:

- The user question
- The source (URL for the paper in PMC)
- The summarized chunks retrieved from the document text
- The generated answer

This is much more comprehensive than the article abstract alone!

### 3.2. Define Agent

Let's define an agent that can use the `search_pmc` tool from notebook 01 and a new `gather_evidence` tool from this notebook to find and understand scientific articles.

In [ ]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence import gather_evidence_tool

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use search_pmc_tool to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 2 years"[dp] for recent work. 
2. Identify the PMC ID value for the 2 relevant papers, then submit their ID values and the query to the gather_evidence_tool.
3. Generate a concise answer to the question based on the most relevant evidence, along with source citations
"""


# Initialize your agent
agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[search_pmc_tool, gather_evidence_tool],
    model=MODEL_ID,
)

# Send a message to the agent
response = agent(QUERY)

## 4. Deploy to Amazon Bedrock AgentCore Runtime

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client("ssm")

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="pmc_gather_evidence_agent",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(
        Name="/deep-research-workshop/agentcore-runtime-role-arn"
    )["Parameter"]["Value"],
    entrypoint="agent.py",
    memory_mode="NO_MEMORY",
    requirements_file="requirements.txt",
)

agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
from invoke_agentcore import invoke_agentcore
import uuid

session_id = str(uuid.uuid4())
invoke_agentcore(
    agent_runtime_name="pmc_gather_evidence_agent",
    prompt="How safe and effective are GLP-1 drugs for long term use?",
    session_id = session_id
)

## 5. (Optional) Interact with agent using a chat application

To experiment with this agent in an instructor-led workshop, follow the steps in the Getting Started section of the Workshop Studio page for this event to access a Streamlit chat application. Then, select `pmc_gather_evidence_agent` from the **Agent Name** list.


## 6. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)